In [1]:
import sys
sys.path.extend(['/Users/igor/Python/CNN-Architecture-Comparison/'])
%load_ext autoreload
%autoreload 2

In [2]:
# ResNet50 - MNIST

In [3]:
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_model.resnet50_network import ResNet50Network

Using TensorFlow backend.


In [4]:
dataset_dir = '/Users/igor/Python/CNN-Architecture-Comparison/data'
fit_epochs = 10

In [5]:
network = ResNet50Network()
network.print_config()

Loss: <function categorical_crossentropy at 0x14cbcb8c0>
Metrics: ['accuracy', <keras.metrics.Precision object at 0x10fed2b50>, <keras.metrics.Recall object at 0x10fed2210>]
Image size: (224, 224)
Grayscale: False
Batch size: 32


In [6]:
dataset_generators = DatasetGenerators(
    name='mnist_10',
    dataset_dir=dataset_dir,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for mnist_10...
Found 4833 images belonging to 10 classes.
Loading test set for mnist_10...
Found 1384 images belonging to 10 classes.
Loading validation set for mnist_10...
Found 695 images belonging to 10 classes.


In [7]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [8]:
history = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test
)

Epoch 1/10
152/152 [==============================] - 1207s 8s/step - loss: 0.6220 - accuracy: 0.8297 - precision_1: 0.8742 - recall_1: 0.8012 - val_loss: 31.3565 - val_accuracy: 0.0990 - val_precision_1: 0.0994 - val_recall_1: 0.0990
Epoch 2/10
 54/152 [=========>....................] - ETA: 11:24 - loss: 0.3389 - accuracy: 0.9087 - precision_1: 0.9137 - recall_1: 0.9051

KeyboardInterrupt: 

In [ ]:
result = model.evaluate_generator(
    generator=dataset_generators.validate
)

## MNIST Results

In [ ]:
print("Loss: {}".format(result[0]))
print("Accuracy: {}".format(result[1]))
print("Precision: {}".format(result[2]))
print("Recall: {}".format(result[3]))